# File aggregation into a single pandas dataset

# Setup

Files needed to run the notebook:

- The zip file *ECJ_FiscalStateAid.zip*; 

In [1]:
!unzip Dataset[1480].zip

unzip:  cannot find or open Dataset[1480].zip, Dataset[1480].zip.zip or Dataset[1480].zip.ZIP.

No zipfiles found.


Import libraries:

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import tensorflow as tf
import os
import json
import numpy as np
from math import isnan

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
path_reject = "Dataset/ECJ_FiscalStateAid JSON/Reject"
path_uphold = "Dataset/ECJ_FiscalStateAid JSON/Uphold"

temp = []

for filename in os.listdir(path_reject):
   with open(os.path.join(path_reject, filename), 'r') as f: # open in readonly mode
      print(filename)
      data = json.load(f)



      annotations = data["annotations"]

      for annotation in annotations:

        document = annotation["document"]
        name = annotation["name"]
        id = annotation["_id"]
        sentence = data["document"]["plainText"][int(annotation["start"]):int(annotation["end"])]
        T= annotation["attributes"].get("T", np.nan)
        SUP = annotation["attributes"].get("SUP", np.nan)
        SFF = annotation["attributes"].get("SFF", np.nan)
        ATT = annotation["attributes"].get("ATT", np.nan)
        INH = annotation["attributes"].get("INH", np.nan)
        REPH = annotation["attributes"].get("REPH", np.nan)
        S = annotation["attributes"].get("S", np.nan)

        temp.append([document,
                      name,
                      id,
                      sentence,
                      T,
                      SUP,
                      SFF,
                      ATT,
                      INH,
                      REPH,
                      S])
        
for filename in os.listdir(path_uphold):
   with open(os.path.join(path_uphold, filename), 'r') as f: # open in readonly mode
      print(filename)
      data = json.load(f)



      annotations = data["annotations"]

      for annotation in annotations:

        document = annotation["document"]
        name = annotation["name"]
        id = annotation["_id"]
        sentence = data["document"]["plainText"][int(annotation["start"]):int(annotation["end"])]
        T= annotation["attributes"].get("T", np.nan)
        SUP = annotation["attributes"].get("SUP", np.nan)
        SFF = annotation["attributes"].get("SFF", np.nan)
        ATT = annotation["attributes"].get("ATT", np.nan)
        INH = annotation["attributes"].get("INH", np.nan)
        REPH = annotation["attributes"].get("REPH", np.nan)
        S = annotation["attributes"].get("S", np.nan)

        temp.append([document,
                      name,
                      id,
                      sentence,
                      T,
                      SUP,
                      SFF,
                      ATT,
                      INH,
                      REPH,
                      S])

df = pd.DataFrame(temp, columns=['Document','Name','Id', 'Sentence', 'Type','Supported_by','Supported_from_failure','Attacked_by', 'Inhibited_by', 'Rephrased_by', 'Argumentation_scheme'])

FileNotFoundError: ignored

# Analysis and Pre-processing

In [61]:
df.head()

,Document,Name,Id,Sentence,Type,Supported_by,Supported_from_failure,Attacked_by,Inhibited_by,Rephrased_by,Argumentation_scheme
0,2004,prem,A1,Both Italy and Wam contest the admissibility o...,F,NaN,NaN,NaN,NaN,NaN,NaN
1,2004,prem,A2,Italy argues that the Commission’s contention ...,F,NaN,NaN,A5,NaN,NaN,NaN
2,2004,prem,A3,"In that respect, it should be borne in mind th...",L,NaN,NaN,NaN,NaN,NaN,Rule
3,2004,prem,A4,As the Advocate General states in point 20 of ...,F,NaN,NaN,NaN,NaN,NaN,Aut
4,2004,prem,A5,It follows that the Italian Republic’s argumen...,F,"[A3, A4]",NaN,NaN,NaN,NaN,NaN


In [62]:
def remove_stopwords(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in  
      string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if 
      word.lower() not in stpword])
df["Sentence"] = df["Sentence"].apply(remove_stopwords)
df["Sentence"] = df["Sentence"].apply(remove_stopwords)

In [63]:
df["Sentence"]

0                  Italy Wam contest admissibility appeal
1       Italy argues Commission’s contention judgment ...
2       respect borne mind pursuant Article 58 Statute...
3       Advocate General states point 20 Opinion Commi...
4       follows Italian Republic’s argument appeal bas...
                              ...                        
1674    must recalled case interpretation national law...
1675    Scuola Elementare Maria Montessori claim disto...
1676    Scuola Elementare Maria Montessori’s argument ...
1677    far Scuola Elementare Maria Montessori additio...
1678           follows second ground appeal must rejected
Name: Sentence, Length: 1679, dtype: object

In [64]:
df['Type'].value_counts(dropna=False)

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


F         921
L         582
NaN       110
L|F        65
[F, L]      1
Name: Type, dtype: int64

In [65]:
#df = df.dropna(subset=['Type']) # Remove rows with NaN in Type column (the rows with NaN are concs)

#df['Type'] = df['Type'].apply(lambda x: 'L|F' if isinstance(x, list) or len(x)>1 else x)
df['Type'] = df['Type'].apply(lambda x: 'L|F' if isinstance(x, list) else x)
df['Type'] = df['Type'].apply(lambda x: 'F' if x=='f' else x)

In [66]:
df['Type'].value_counts(dropna=False)

F      921
L      582
NaN    110
L|F     66
Name: Type, dtype: int64

In [67]:
df['Document'].value_counts(dropna=False)

2028    109
2001    106
2007     95
2011     93
2027     91
2003     77
2000     73
2012     71
2005     68
2022     65
2016     61
2023     59
2015     56
2013     50
2009     50
2019     50
2017     50
2008     48
2010     45
2020     44
2014     43
2021     40
2006     40
2025     38
2018     37
2004     37
2024     30
2026     29
2002     24
Name: Document, dtype: int64

In [68]:
df['Name'].value_counts(dropna=False)

prem    1569
conc     110
Name: Name, dtype: int64

In [69]:
df['Name'] = df['Name'].apply(lambda x: 'prem' if x=='perm' else x)

In [70]:
df['Name'].value_counts(dropna=False)

prem    1569
conc     110
Name: Name, dtype: int64

In [71]:
df.head()

,Document,Name,Id,Sentence,Type,Supported_by,Supported_from_failure,Attacked_by,Inhibited_by,Rephrased_by,Argumentation_scheme
0,2004,prem,A1,Italy Wam contest admissibility appeal,F,NaN,NaN,NaN,NaN,NaN,NaN
1,2004,prem,A2,Italy argues Commission’s contention judgment ...,F,NaN,NaN,A5,NaN,NaN,NaN
2,2004,prem,A3,respect borne mind pursuant Article 58 Statute...,L,NaN,NaN,NaN,NaN,NaN,Rule
3,2004,prem,A4,Advocate General states point 20 Opinion Commi...,F,NaN,NaN,NaN,NaN,NaN,Aut
4,2004,prem,A5,follows Italian Republic’s argument appeal bas...,F,"[A3, A4]",NaN,NaN,NaN,NaN,NaN


In [72]:
df[df["Name"] == 'prem']['Argumentation_scheme'].shape

(1569,)

In [ ]:
df[df["Name"] == 'prem']['Argumentation_scheme'].value_counts(dropna=False) # To preserve conc rows

In [74]:
scheme = ["Aut", "Class", "Itpr", "Lit", "Prec", "Princ", "Psy", "Rule", "Syst", "Tele"]

#df = df.dropna(subset=['Argumentation_scheme']) # Remove rows with NaN in Argumentation_scheme column
nan = np.nan
df = df.query("Name == 'conc' or Argumentation_scheme == Argumentation_scheme")

df["Argumentation_scheme"] = df["Argumentation_scheme"].apply(lambda x: [x[i] for i in range(len(x)) if x[i] in scheme] if isinstance(x, list) else x)
df["Argumentation_scheme"] = df["Argumentation_scheme"].apply(lambda x: "|".join(sorted(x)) if isinstance(x, list) else x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
df["Name"].value_counts()

prem    663
conc    110
Name: Name, dtype: int64

In [76]:
df['Argumentation_scheme'].value_counts(dropna=False)

Prec                  279
NaN                   110
Itpr                   95
Princ                  64
Rule                   61
Prec|Rule              35
Itpr|Prec              22
Aut                    21
Class|Prec|Rule         9
Tele                    8
Class                   7
Prec|Tele               5
Class|Prec              5
Aut|Itpr                5
Itpr|Tele               4
Rule|Syst               3
Syst                    3
Itpr|Rule               3
Prec|Syst               3
Psy|Tele                2
Lit|Prec                2
Princ|Rule              2
Lit|Rule                2
Prec|Psy|Rule|Tele      1
Syst|Tele               1
Princ|Tele              1
Itpr|Syst               1
Itpr|Psy                1
Aut|Prec|Princ          1
Itpr|Lit                1
Lit|Rule|Syst           1
Princ|Psy|Syst          1
Itpr|Princ|Syst         1
Aut|Class               1
Lit|Rule|Tele           1
Aut|Itpr|Prec           1
Prec|Psy                1
Aut|Rule                1
Aut|Syst    

In [77]:
df = df.drop(df[df['Argumentation_scheme'] == "Rule???"].index)
df = df.drop(df[df['Argumentation_scheme'] == "B3"].index)
df = df.drop(df[df['Argumentation_scheme'] == "XXX"].index)
df = df.drop(df[df['Argumentation_scheme'] == "Syst???"].index)
df = df.drop(df[df['Argumentation_scheme'] == "Prec>"].index)
df = df.drop(df[df['Argumentation_scheme'] == "C3"].index)

In [78]:
df[df["Name"] == 'prem']['Argumentation_scheme'].value_counts(dropna=False)

Prec                  279
Itpr                   95
Princ                  64
Rule                   61
Prec|Rule              35
Itpr|Prec              22
Aut                    21
Class|Prec|Rule         9
Tele                    8
Class                   7
Class|Prec              5
Aut|Itpr                5
Prec|Tele               5
Itpr|Tele               4
Rule|Syst               3
Itpr|Rule               3
Prec|Syst               3
Syst                    3
Princ|Rule              2
Psy|Tele                2
Lit|Rule                2
Lit|Prec                2
Prec|Psy|Rule|Tele      1
Princ|Tele              1
Itpr|Syst               1
Itpr|Psy                1
Itpr|Lit                1
Aut|Prec|Princ          1
Princ|Psy|Syst          1
Itpr|Princ|Syst         1
Lit|Rule|Syst           1
Syst|Tele               1
Aut|Class               1
Lit|Rule|Tele           1
Aut|Itpr|Prec           1
Prec|Psy                1
Aut|Rule                1
Aut|Syst                1
Lit         

In [79]:
df.to_pickle("./dataset.pkl")  